In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/cat-in-the-dat/sample_submission.csv
/kaggle/input/cat-in-the-dat/train.csv
/kaggle/input/cat-in-the-dat/test.csv


In [42]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.decomposition import TruncatedSVD

import string
import itertools

In [43]:
submission = pd.read_csv("/kaggle/input/cat-in-the-dat/sample_submission.csv")
train = pd.read_csv("/kaggle/input/cat-in-the-dat/train.csv")
test = pd.read_csv("/kaggle/input/cat-in-the-dat/test.csv")



In [44]:
# getting train test data id's
train_id = train['id']
test_id = test['id']
target = train['target']

In [45]:
# merging train test data sets into one data frame for feature engineering 
df = pd.concat([train.drop(labels=['id','target'],axis=1), test.drop(labels='id',axis=1)])

In [46]:
len(df.columns.values)

23

In [47]:
train.shape

(300000, 25)

In [48]:

pre = ['bin_0','bin_1','bin_2','bin_3','bin_4','nom_0','nom_1','nom_2','nom_3','nom_4','nom_5','nom_6','nom_7','nom_8','nom_9','ord_0','ord_1','ord_2','ord_3','ord_4','ord_5','day','month']
type(pre)

list

In [49]:

df_feat= df.columns.values.tolist()

In [50]:
# Because of varity of features, data set divided into 4 dataframes for analyzing 

bin_feat = df[['bin_0','bin_1','bin_2','bin_3','bin_4']]
nom_feat = df[['nom_0','nom_1','nom_2','nom_3','nom_4','nom_5','nom_6','nom_7','nom_8','nom_9']]
ord_feat = df[['ord_0','ord_1','ord_2','ord_3','ord_4','ord_5']]
cyc_feat = df[['day','month']]

In [51]:
bin_feat['bin_3'] = bin_feat['bin_3'].map({'T':1,'F':0})
bin_feat['bin_4'] = bin_feat['bin_4'].map({'Y':1,'N':0})

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
bin_feat.head()

,bin_0,bin_1,bin_2,bin_3,bin_4
0,0,0,0,1,1
1,0,1,0,1,1
2,0,0,0,0,1
3,0,1,0,0,1
4,0,0,0,0,0


In [53]:
bin_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 199999
Data columns (total 5 columns):
bin_0    500000 non-null int64
bin_1    500000 non-null int64
bin_2    500000 non-null int64
bin_3    500000 non-null int64
bin_4    500000 non-null int64
dtypes: int64(5)
memory usage: 22.9 MB


In [54]:
nom_feat.head()

,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51
1,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21
2,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0
3,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71
4,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7


In [55]:
num_nom = nom_feat.apply(pd.Series.value_counts)

In [56]:
# counting the number of nominal features
num_nom.shape

(16276, 10)

# Using sparse dataframe

In [57]:
label_en = LabelEncoder()

In [58]:
nom_feat = nom_feat.apply(LabelEncoder().fit_transform)

In [59]:
nom_feat.head()

,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,1,5,5,3,0,78,120,491,1689,2192
1,1,4,3,5,2,159,510,260,652,11719
2,0,4,4,5,3,44,14,766,1935,8140
3,2,4,5,0,1,209,165,1121,631,6098
4,2,4,4,0,1,90,61,34,1763,8293


In [60]:
one_nom = OneHotEncoder(handle_unknown='ignore')

In [61]:
one_nom.fit(nom_feat)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [62]:
fet = one_nom.transform(nom_feat)

In [63]:
fet.shape

(500000, 16276)

In [64]:
svd = TruncatedSVD(n_components=10, n_iter=5, random_state=42)


In [65]:
svd.fit(fet)

TruncatedSVD(algorithm='randomized', n_components=10, n_iter=5, random_state=42,
             tol=0.0)

In [66]:
%%time
nom_svd = svd.transform(fet)

CPU times: user 92 ms, sys: 36 ms, total: 128 ms
Wall time: 126 ms


In [ ]:
#nom_svd

In [67]:
nom_svd_df = pd.DataFrame(data=nom_svd,columns = ['nom_0','nom_1','nom_2','nom_3','nom_4','nom_5','nom_6','nom_7','nom_8','nom_9'] )

In [68]:
nom_svd_df.head()

,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,0.912354,-0.721664,-0.107128,-0.249346,-0.090145,-0.028202,0.369176,0.897247,-0.343402,0.360899
1,1.404079,-0.583058,-0.725255,0.035262,-0.807097,0.027206,-0.084431,-0.731002,-0.206743,0.072109
2,1.430693,0.814660,-0.171870,-0.225055,-0.001537,0.000745,0.884795,-0.862193,0.832397,-0.000655
3,1.088971,0.323848,0.750666,0.852795,-0.445171,-0.829210,-0.395617,0.145207,-0.067756,0.981278
4,1.299651,0.377088,0.731816,0.952230,0.485980,-0.837179,-0.060686,-0.355357,-0.114486,0.519720


In [69]:
train_nom = nom_svd_df.iloc[0:300000]
test_nom = nom_svd_df.iloc[300000:]

In [70]:
train_nom_y = train['target']

In [71]:
train_nom_y.value_counts()

0    208236
1     91764
Name: target, dtype: int64

In [82]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

In [83]:
logit = LogisticRegression(solver='lbfgs', max_iter=10000, cv = KFold(n_splits=10, random_state=42)).fit(train_nom,train_nom_y)

TypeError: __init__() got an unexpected keyword argument 'cv'

In [74]:
pred = logit.predict_proba(train_nom)

In [75]:
pred_v = logit.predict(train_nom)

In [78]:
from sklearn.metrics import confusion_matrix,roc_auc_score,auc,accuracy_score

In [81]:
accuracy_score(train_nom_y,pred_v)

0.69412